# Shallow Moment Tutorial (Simple)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import jax
from jax import numpy as jnp
import pytest
from types import SimpleNamespace
from sympy import cos, pi, Piecewise

from library.fvm.solver import HyperbolicSolver, Settings
from library.fvm.ode import RK1
import library.fvm.reconstruction as recon
import library.fvm.timestepping as timestepping
import library.fvm.flux as flux
import library.fvm.nonconservative_flux as nc_flux
from library.model.boundary_conditions import BoundaryCondition
from library.model.models.basisfunctions import Basisfunction, Legendre_shifted
from library.model.models.basismatrices import Basismatrices
from library.misc.misc import Zstruct

from library.model.models.shallow_moments import ShallowMoments2d
import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
import library.misc.io as io
from library.mesh.mesh import compute_derivatives
from tests.pdesoft import plots_paper
import library.postprocessing.visualization as visu


import library.mesh.mesh as petscMesh
import library.postprocessing.postprocessing as postprocessing
from library.mesh.mesh import convert_mesh_to_jax
import argparse

2025-08-08 06:21:29.819 | WARNING  | library.fvm.solver:<module>:28 - No module named 'precice'


## Model

In [3]:
level = 0
offset = 1+level
n_fields = 3 + 2 * level
settings = Settings(
    name="SME",
    output=Zstruct(
        directory=f"outputs/junction_{level}", filename="SME", output_snapshots=30
    ),
)

2025-08-08 06:21:39.648 | WARNING  | library.misc.misc:__init__:136 - No 'solver' Zstruct found in Settings. Default: empty Zstruct
2025-08-08 06:21:39.649 | WARNING  | library.misc.misc:__init__:139 - No 'model' Zstruct found in Settings. Default: empty Zstruct
2025-08-08 06:21:39.649 | WARNING  | library.misc.misc:__init__:153 - No 'clean_directory' attribute found in output Zstruct. Default: False
2025-08-08 06:21:39.649 | WARNING  | library.misc.misc:__init__:156 - No 'parameters' attribute found in model Zstruct. Default: empty Zstruct


In [6]:
inflow_dict = { 
    0: lambda t, x, dx, q, qaux, p, n: Piecewise((0.1, t < 0.2),(q[0], True)),
    1: lambda t, x, dx, q, qaux, p, n: Piecewise((-0.3, t < 0.2),(-q[1], True)),
                }
inflow_dict.update({
    1+i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level)
})
inflow_dict.update({
    1+offset+i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level+1)
})

bcs = BC.BoundaryConditions(
    [
        BC.Lambda(physical_tag="inflow", prescribe_fields=inflow_dict),
        BC.Wall(physical_tag="wall"),
    ]
)

def custom_ic(x):
    Q = np.zeros(3 + 2 * level, dtype=float)
    Q[0] = 0.01
    return Q

ic = IC.UserFunction(custom_ic)

model = ShallowMoments2d(
    level=level,
    boundary_conditions=bcs,
    initial_conditions=ic,
)

main_dir = os.getenv("SMS")
mesh = petscMesh.Mesh.from_gmsh(
    os.path.join(main_dir, "meshes/channel_junction/mesh_2d_coarse.msh")
    # os.path.join(main_dir, "meshes/channel_junction/mesh_2d_fine.msh")
)

mesh = convert_mesh_to_jax(mesh)
class SMESolver(HyperbolicSolver):
    def update_qaux(self, Q, Qaux, Qold, Qauxold, mesh, model, parameters, time, dt):
        dudx = compute_derivatives(Q[1]/Q[0], mesh, derivatives_multi_index=[[0, 0]])[:,0]
        dvdy = compute_derivatives(Q[1+offset]/Q[0], mesh, derivatives_multi_index=[[0, 1]])[:,0]
        Qaux = Qaux.at[0].set(dudx)
        Qaux = Qaux.at[1].set(dvdy)
        return Qaux
solver = SMESolver(settings=settings)

## Solve

In [7]:
Qnew, Qaux = solver.solve(mesh, model)

2025-08-08 06:22:47.778 | INFO     | library.fvm.solver:log_callback_hyperbolic:43 - iteration: 1, time: 0.006773, dt: 0.006773, next write at time: 0.011111
2025-08-08 06:22:47.797 | INFO     | library.fvm.solver:log_callback_hyperbolic:43 - iteration: 2, time: 0.007304, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:22:47.817 | INFO     | library.fvm.solver:log_callback_hyperbolic:43 - iteration: 3, time: 0.007836, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:22:47.841 | INFO     | library.fvm.solver:log_callback_hyperbolic:43 - iteration: 4, time: 0.008368, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:22:47.858 | INFO     | library.fvm.solver:log_callback_hyperbolic:43 - iteration: 5, time: 0.008899, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:22:47.879 | INFO     | library.fvm.solver:log_callback_hyperbolic:43 - iteration: 6, time: 0.009431, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:22:47.897 | INFO     | library.fvm.sol

## Visualization

In [8]:
io.generate_vtk(os.path.join(settings.output.directory, f"{settings.output.filename}.h5"))
postprocessing.vtk_interpolate_3d(model, settings, Nz=20, filename='out_3d')

2025-08-08 06:22:58.053 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 0/10
2025-08-08 06:22:58.079 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 1/10
2025-08-08 06:22:58.102 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 2/10
2025-08-08 06:22:58.125 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 3/10
2025-08-08 06:22:58.151 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 4/10
2025-08-08 06:22:58.179 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 5/10
2025-08-08 06:22:58.207 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 6/10
2025-08-08 06:22:58.233 | INFO     | library.postprocessing.postprocessing:vtk_interpolate_3d:62 - Converted snapshot 7/10
2025-08-08 06:22

In [9]:
visu.pyvista_3d(settings.output.directory, scale=1.0)

BokehModel(combine_events=True, render_bundle={'docs_json': {'27d40e6f-9809-46fc-aaa7-e2c427429dcd': {'version…